In [ ]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split

# Define the path to your CSV file
# csv_file = r'/kaggle/input/clothing-dataset-full/images.csv'
csv_file = r'Clothes/Clothing dataset full/images.csv'

# Define the base directory where you want to create class-specific subdirectories
# base_directory = '/kaggle/input/clothing-dataset-full/images_original'  # Replace with your desired directory
base_directory = 'Clothes/Clothing dataset full/images_original'

# Define destination directory for the entire dataset
# dest_dir = '/kaggle/working/new_dataset'
dest_dir = 'Clothes/Clothing dataset full/new_dataset'

# Load the CSV file
data = pd.read_csv(csv_file)

# Define directories for training, validation, and test sets
# train_dir = '/kaggle/working/train_dataset'
# val_dir = '/kaggle/working/val_dataset'
# test_dir = '/kaggle/working/test_dataset'

train_dir = 'Clothes/Clothing dataset full/train_dataset'
val_dir = 'Clothes/Clothing dataset full/val_dataset'
test_dir = 'Clothes/Clothing dataset full/test_dataset'

# Create directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Split data into training (70%), validation (15%), and test (15%) sets
train_data, temp_data = train_test_split(data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)


# Function to move images to specific directories
def move_images(data, dest_dir):
    for index, row in data.iterrows():
        image_path = row['image'] + ".jpg"
        class_name = row['label']
        src_path = os.path.join(base_directory, image_path)
        dest_path = os.path.join(dest_dir, class_name)
        
        # Ensure the destination directory exists
        os.makedirs(dest_path, exist_ok=True)

        # Copy the image to the destination directory
        if os.path.exists(src_path):
            shutil.copy(src_path, dest_path)

# Move images to the training set
move_images(train_data, train_dir)

# Move images to the validation set
move_images(val_data, val_dir)

# Move images to the test set
move_images(test_data, test_dir)


**Borrar carpeta Skip**

In [ ]:
import shutil
import os

# folder_path = '/kaggle/working/train_dataset/Skip'
folder_path = 'Clothes/Clothing dataset full/train_dataset/Skip'

if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print(f"{folder_path} and its contents have been deleted.")
else:
    print(f"{folder_path} does not exist.")

In [ ]:
import shutil
import os

# folder_path = '/kaggle/working/test_dataset/Skip'
folder_path = 'Clothes/Clothing dataset full/test_dataset/Skip'

if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print(f"{folder_path} and its contents have been deleted.")
else:
    print(f"{folder_path} does not exist.")

In [ ]:
import shutil
import os

# folder_path = '/kaggle/working/val_dataset/Skip'
folder_path = 'Clothes/Clothing dataset full/val_dataset/Skip'

if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print(f"{folder_path} and its contents have been deleted.")
else:
    print(f"{folder_path} does not exist.")

**Borrar carpeta Not Sure**

In [ ]:
import shutil
import os

# folder_path = '/kaggle/working/val_dataset/Not sure'
folder_path = 'Clothes/Clothing dataset full/val_dataset/Not sure'

if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print(f"{folder_path} and its contents have been deleted.")
else:
    print(f"{folder_path} does not exist.")

In [ ]:
import shutil
import os

# folder_path = '/kaggle/working/test_dataset/Not sure'
folder_path = 'Clothes/Clothing dataset full/test_dataset/Not sure'

if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print(f"{folder_path} and its contents have been deleted.")
else:
    print(f"{folder_path} does not exist.")

In [ ]:
import shutil
import os

# folder_path = '/kaggle/working/train_dataset/Not sure'
folder_path = 'Clothes/Clothing dataset full/train_dataset/Not sure'

if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print(f"{folder_path} and its contents have been deleted.")
else:
    print(f"{folder_path} does not exist.")

## **Creación del modelo**

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization

# Directorios de los conjuntos de datos
# train_dir = '/kaggle/working/train_dataset'
# val_dir = '/kaggle/working/val_dataset'

train_dir = 'Clothes/Clothing dataset full/train_dataset'
val_dir = 'Clothes/Clothing dataset full/val_dataset'

# Cargar el modelo VGG16 con pesos preentrenados de ImageNet
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelar los pesos del modelo base
for layer in base_model.layers:
    layer.trainable = False

# Añadir nuevas capas
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
predictions = Dense(18, activation='softmax')(x)

# Crear el modelo final
model = Model(inputs=base_model.input, outputs=predictions)

# Compilar el modelo
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Configuración del generador de imágenes
train_datagen = ImageDataGenerator(
    rescale=1./255,
    # Agregar más transformaciones si es necesario
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Preparar los generadores de datos
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

In [ ]:
# Entrenar el modelo
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // validation_generator.batch_size
)

In [ ]:
model.save('classifier_model_20EPOCH_VGG16.h5')

#  **Generador de MATRIZ**

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Importante para mantener el orden para comparar con las etiquetas verdaderas
)

# Hacer predicciones
predicciones = model.predict(test_generator)


In [ ]:
import numpy as np
y_pred = np.argmax(predicciones, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Etiquetas verdaderas
y_true = test_generator.classes

# Crear la matriz de confusión
matriz_confusion = confusion_matrix(y_true, y_pred)

# Visualizar la matriz de confusión
plt.figure(figsize=(10, 10))
sns.heatmap(matriz_confusion, annot=True, fmt="d", linewidths=.5, square=True)
plt.xlabel('Predicciones')
plt.ylabel('Valores verdaderos')
plt.title('Matriz de Confusión')
plt.savefig('matriz_confusion.jpg')
plt.show()


# # **Predecir Imagen**

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Ruta de la imagen que deseas clasificar
image_path = '/kaggle/working/val_dataset/Hoodie/3975a9b1-ef23-4688-95f8-3500c1850c1b.jpg'  # Reemplaza con la ruta de tu imagen

# Cargar y preprocesar la imagen
img = image.load_img(image_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # Normalizar la imagen si es necesario

In [ ]:
# Realizar la predicción
predictions = model.predict(img_array)

# Obtener la etiqueta predicha
class_labels = list(train_generator.class_indices.keys()) # Reemplaza con las clases de tu modelo
predicted_class = class_labels[np.argmax(predictions)]

print("Clase predicha:", predicted_class)


In [ ]:
# Obtener la etiqueta verdadera (extraída del nombre de la carpeta en la ruta)
true_class = image_path.split('/')[-2]

# Ajustar el ancho de la figura para la gráfica de barras
plt.figure(figsize=(14, 6))

# Mostrar la imagen (tamaño normal)
plt.subplot(1, 2, 1)
img = mpimg.imread(image_path)
plt.imshow(img)
plt.axis('off')
plt.title('Imagen de Entrada\n(Clase Verdadera: {})'.format(true_class))

# Mostrar la gráfica de barras de las probabilidades predichas (más ancho)
plt.subplot(1, 2, 2)
plt.bar(class_labels, predictions.flatten(), color='blue')
plt.xlabel('Clases')
plt.ylabel('Probabilidades')
plt.title('Probabilidades Predichas')
plt.xticks(rotation=45, ha='right')

plt.show()